## Observation Maker from QR

1.  Enter source data in QR file to simulate patient generated  observations


Patient use existing US Core patient:

- Add example name and description
- Validate
- Save

In [1]:
%cd '/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker'

/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker


In [2]:
from fhir.resources.observation import Observation
from fhir.resources.questionnaireresponse import QuestionnaireResponse
from fhir.resources.quantity import Quantity
from fhir.resources import construct_fhir_element
from pathlib import Path
from datetime import datetime, date, timezone, timedelta
from stringcase import snakecase, titlecase, spinalcase
from requests import post
from IPython.display import display as Display, HTML

timezone_offset = -8.0  # Pacific Standard Time (UTC−08:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
f_now = datetime.now(tzinfo)

#ig_source_path = 'examples'
ig_source_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/input/examples'

### Write to file Function

In [3]:
def write(obs_obj):

    path = Path.cwd() / ig_source_path / f'{obs_obj.id}.json'   
    print(f'...........saving to file {path}............')
    path.write_text(obs_obj.json(indent=2))

### Validate  Function

In [4]:
def validate(data):
    
    print('...validating')

    fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    r = post(f'{fhir_test_server}/Observation/$validate', headers = headers, data = data)
    r.status_code, r.json()
    Display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

###  Import completed QR YAML file

In [6]:
qr = 'bone_qr' #'cardio_qr' #"Observation_wt.yml")
qr_obj = QuestionnaireResponse.parse_file(f"source_data/{qr}.yml")
print(qr_obj.yaml())

resourceType: QuestionnaireResponse
id: '5770426'
meta:
  versionId: '1'
  lastUpdated: 2021-11-10 22:02:19-05:00
  source: '#UgU1K2o2WfbL2QDl'
  profile:
  - http://hl7.org/fhir/uv/sdc/StructureDefinition/sdc-questionnaireresponse|2.7
  tag:
  - code: 'lformsVersion: 29.1.0'
questionnaire: Questionnaire/5770424
status: completed
subject:
  reference: Patient/pat-12346
  display: LEIGH AMIN
authored: 2021-11-11 03:02:15.888000+00:00
item:
- linkId: /80954-1
  text: DXA Femur-L ArMass BMD
  answer:
  - valueDecimal: 0.631
- linkId: /80948-3
  text: DXA Femur-L T-score BMD
  answer:
  - valueDecimal: -1.48
- linkId: /80940-0
  text: DXA Femur-L Z-score BMD
  answer:
  - valueDecimal: 0.05
- linkId: /46278-8
  text: DXA Hip-L ArMass BMD
  answer:
  - valueDecimal: 0.734
- linkId: /80946-7
  text: DXA Hip-L T-score BMD
  answer:
  - valueDecimal: -1.62
- linkId: /80938-4
  text: DXA Hip-L Z-score BMD
  answer:
  - valueDecimal: -0.19



###  Import from YAML file Observation template and fill in with QR values

In [9]:
def kebab(s):
    '''
    change to kebab case
    '''
    s=s.replace('(','').replace(')','').replace('{','').replace('}','')
    s_list = s.lower().split()
    s = '-'.join(s_list)
    return(s)

for i in qr_obj.item:
    obs_obj = Observation.parse_file("fhir_templates/Obs_clintest.yml")#"Observation_wt.yml")
    obs_obj.encounter.display = "Bone Scan"
    print(obs_obj.yaml())
    print((i.text), kebab(i.text))
    obs_obj.id = kebab(i.text)
    obs_obj.effectiveDateTime = f_now
    obs_obj.meta.extension[0].valueString = f'{i.text} Example'
    obs_obj.meta.extension[1].valueMarkdown = f'This is a {i.text} Example for the *Clinical Test Result Observation Profile*.'
    cat = input(f"What category is {i.text}?  exam|procedure|activity|imaging?")
    obs_obj.category[0].coding[0].code = cat
    obs_obj.category[0].coding[0].display = cat.capitalize()
    obs_obj.category[0].text = cat.capitalize()
    obs_obj.code.coding[0].code = i.linkId[1:]
    obs_obj.code.coding[0].display = i.text
    obs_obj.code.text = i.text
    
    if i.answer[0].valueCoding:
        obs_obj.valueCodeableConcept.coding[0].code = i.answer[0].valueCoding.code
        obs_obj.valueCodeableConcept.coding[0].display = i.answer[0].valueCoding.display
        obs_obj.valueCodeableConcept.text = i.answer[0].valueCoding.display
    elif i.answer[0].valueString:
        obs_obj.valueCodeableConcept=None
        obs_obj.valueString = i.answer[0].valueString
    else: # i.answer[0].valueDecimal
        obs_obj.valueCodeableConcept=None
        quant = construct_fhir_element('Quantity', {})
        quant.value = i.answer[0].valueDecimal
        quant.unit = input(f"What is the unit for {i.text}? s|ms?")
        quant.system = "http://unitsofmeasure.org"
        score = input(f"is this a score Y/N ?")
        if score == 'Y':
            quant.code = f"{{{quant.unit}}}"
        else:
            quant.code = quant.unit
        print(quant.yaml())
        obs_obj.valueQuantity=quant
    print(obs_obj.yaml())
    validate(obs_obj.json())
    write(obs_obj)


resourceType: Observation
id: pulse-intensity-palpation
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: Pulse intensity Palpation Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a Pulse intensity Palpation Example for the *Clinical Test
      Result Observation Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: exam
    display: Exam
  text: Exam
code:
  coding:
  - system: http://loinc.org
    code: 44974-4
    display: Pulse intensity Palpation
  text: Pulse intensity Palpation
subject:
  reference: Patient/example
  display: Amy Shaw
encounter:
  display: Bone Scan
effectiveDateTime: 2021-11-10
valueCodeableConcept:
  coding:
  - system: http://loinc.org
    code: LA11841-6
    display: 1+
  text: 1+

DXA Fem

What category is DXA Femur-L ArMass BMD?  exam|procedure|activity|imaging? imaging
What is the unit for DXA Femur-L ArMass BMD? s|ms? g/cm2
is this a score Y/N ? n


value: 0.631
unit: g/cm2
system: http://unitsofmeasure.org
code: g/cm2

resourceType: Observation
id: dxa-femur-l-armass-bmd
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: DXA Femur-L ArMass BMD Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a DXA Femur-L ArMass BMD Example for the *Clinical Test
      Result Observation Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: imaging
    display: Imaging
  text: Imaging
code:
  coding:
  - system: http://loinc.org
    code: 80954-1
    display: DXA Femur-L ArMass BMD
  text: DXA Femur-L ArMass BMD
subject:
  reference: Patient/example
  display: Amy Shaw
encounter:
  display: Bone Scan
effectiveDateTime: 2021-11-10 19:14:41.088720-08:00
valueQuantity:
  value: 0.63

...........saving to file /Users/ehaas/Documents/FHIR/US-Core-R4/input/examples/dxa-femur-l-armass-bmd.json............
resourceType: Observation
id: pulse-intensity-palpation
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: Pulse intensity Palpation Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a Pulse intensity Palpation Example for the *Clinical Test
      Result Observation Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: exam
    display: Exam
  text: Exam
code:
  coding:
  - system: http://loinc.org
    code: 44974-4
    display: Pulse intensity Palpation
  text: Pulse intensity Palpation
subject:
  reference: Patient/example
  display: Amy Shaw
encounter:
  display: Bone Scan
effectiveDateTime: 2021-11

What category is DXA Femur-L T-score BMD?  exam|procedure|activity|imaging? imaging
What is the unit for DXA Femur-L T-score BMD? s|ms? T-Score
is this a score Y/N ? Y


...

value: -1.62
unit: T-Score
system: http://unitsofmeasure.org
code: '{T-Score}'

resourceType: Observation
id: dxa-hip-l-t-score-bmd
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: DXA Hip-L T-score BMD Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a DXA Hip-L T-score BMD Example for the *Clinical Test
      Result Observation Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: imaging
    display: Imaging
  text: Imaging
code:
  coding:
  - system: http://loinc.org
    code: 80946-7
    display: DXA Hip-L T-score BMD
  text: DXA Hip-L T-score BMD
subject:
  reference: Patient/example
  display: Amy Shaw
encounter:
  display: Bone Scan
effectiveDateTime: 2021-11-10 19:14:41.088720-08:00
valueQuantity:
  value: -

...........saving to file /Users/ehaas/Documents/FHIR/US-Core-R4/input/examples/dxa-hip-l-t-score-bmd.json............
resourceType: Observation
id: pulse-intensity-palpation
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: Pulse intensity Palpation Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a Pulse intensity Palpation Example for the *Clinical Test
      Result Observation Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: exam
    display: Exam
  text: Exam
code:
  coding:
  - system: http://loinc.org
    code: 44974-4
    display: Pulse intensity Palpation
  text: Pulse intensity Palpation
subject:
  reference: Patient/example
  display: Amy Shaw
encounter:
  display: Bone Scan
effectiveDateTime: 2021-11-

What category is DXA Hip-L Z-score BMD?  exam|procedure|activity|imaging? imaging
What is the unit for DXA Hip-L Z-score BMD? s|ms? Z-Score
is this a score Y/N ? Y


value: -0.19
unit: Z-Score
system: http://unitsofmeasure.org
code: '{Z-Score}'

resourceType: Observation
id: dxa-hip-l-z-score-bmd
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: DXA Hip-L Z-score BMD Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a DXA Hip-L Z-score BMD Example for the *Clinical Test
      Result Observation Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test
status: final
category:
- coding:
  - system: http://terminology.hl7.org/CodeSystem/observation-category
    code: imaging
    display: Imaging
  text: Imaging
code:
  coding:
  - system: http://loinc.org
    code: 80938-4
    display: DXA Hip-L Z-score BMD
  text: DXA Hip-L Z-score BMD
subject:
  reference: Patient/example
  display: Amy Shaw
encounter:
  display: Bone Scan
effectiveDateTime: 2021-11-10 19:14:41.088720-08:00
valueQuantity:
  value: -

...........saving to file /Users/ehaas/Documents/FHIR/US-Core-R4/input/examples/dxa-hip-l-z-score-bmd.json............
